Here's a separate script that fetches historical stock data and implied volatility using yfinance, then creates a dataset for 15 options with randomly selected strikes and expirations. This script will save the data to historical_options_data.csv for use in the LSTM Black-Scholes integration.

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import random

# Define stock ticker and fetch data
ticker = "AAPL"  # Change to any stock symbol
stock = yf.Ticker(ticker)

# Fetch historical stock prices
hist = stock.history(period="90d")  # Last 90 days of stock data
hist['Date'] = hist.index

# Fetch options expirations
expirations = stock.options
if not expirations:
    raise ValueError("No options data available for this ticker.")

# Select 15 random expiration dates
selected_expirations = random.sample(expirations, min(15, len(expirations)))

# Initialize data storage
option_data = []

# Fetch option chains and extract details
for exp in selected_expirations:
    opt_chain = stock.option_chain(exp)
    calls = opt_chain.calls  # Get call options

    # Select a random call option
    if not calls.empty:
        call_option = calls.sample(n=1)  # Pick one random call per expiration
        S = hist.iloc[-1]["Close"]  # Latest stock price
        K = call_option["strike"].values[0]  # Strike price
#        T = (pd.to_datetime(exp) - pd.to_datetime(hist.iloc[-1]["Date"])).days / 365  # Time to expiration in years
        T = (pd.to_datetime(exp).tz_localize(None) - pd.to_datetime(hist.iloc[-1]["Date"]).tz_localize(None)).days / 365

        r = 0.05  # Assume a 5% risk-free rate
        sigma = call_option["impliedVolatility"].values[0]  # Implied volatility

        option_data.append([S, K, T, r, sigma])

# Convert to DataFrame and save to CSV
df = pd.DataFrame(option_data, columns=["S", "K", "T", "r", "sigma"])
df.to_csv("historical_options_data.csv", index=False)

print("Saved historical_options_data.csv with 15 options.")


Saved historical_options_data.csv with 15 options.


How This Script Works
Fetches historical stock prices for the past 90 days.
Retrieves available option expiration dates for the given stock.
Selects 15 random call options from different expirations.
Extracts:
S (Stock Price)
K (Strike Price)
T (Time to Expiration in Years)
r (Risk-Free Rate, assumed 5%)
σ (Implied Volatility)
Saves the dataset to historical_options_data.csv.
Next Steps
Now you can run this script first to generate historical_options_data.csv.
Then, use the LSTM Black-Scholes integration script to train and price options.

Next Steps
Now you can run this script first to generate historical_options_data.csv.
Then, use the LSTM Black-Scholes integration script to train and price options.

Would you like to modify this for puts, different risk-free rates, or real-time updating?